In [1]:
!pip install ns3
!pip install pyshark
!apt-get install tshark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 22.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cppyy_backend-1.14.10-py2.py3-none-linux_x86_64.whl
  Using cached cppyy_cling-6.27.1-py2.py3-none-manylinux2014_x86_64.whl (32.4 MB)
  Created wheel for cppyy: filename=cppyy-2.4.2-py3-none-any.whl size=15352126 sha256=77ca4dbfd8f51b32ab07e1e972cdbdac0877041d496eb791b332bbe618507bf9
  Stored in directory: /root/.cache/pip/wheels/73/66/49/38da85078450bb1a5b63405aec0935de2e2991717394ac2f64
  Created wheel for CPyCppyy: filename=CPyCppyy-1.12.12-cp310-cp310-linux_x86_64.whl size=4182313 sha256=b197e1997f2a

In [2]:
from ns import ns

In [ ]:
nodes = ns.NodeContainer()
nodes.Create(2)

pointToPoint = ns.PointToPointHelper()
pointToPoint.SetDeviceAttribute("DataRate", ns.StringValue("5Mbps"))
pointToPoint.SetChannelAttribute("Delay", ns.StringValue("2ms"))

devices = pointToPoint.Install(nodes)
pointToPoint.EnablePcapAll("./application", True)

stack = ns.InternetStackHelper()
stack.Install(nodes)

address = ns.Ipv4AddressHelper()
address.SetBase(ns.Ipv4Address("10.1.1.0"),
                ns.Ipv4Mask("255.255.255.0"))

interfaces = address.Assign(devices)

In [ ]:
# We need to use this weird trick of defining c++ stuff from Python because
# template forwarding isn't supported by our Cppyy bindings.

ns.cppyy.cppdef("""
    #ifndef AppCallbackAndSend
    #define AppCallbackAndSend
    namespace ns3
    {
        Callback<void,Ptr<Socket> > make_rx_callback(void(*func)(Ptr<Socket>))
        {
            return MakeCallback(func);
        }
        EventImpl* pythonMakeEventSend(void (*f)(Ptr<Socket>, Ptr<Packet>, Address&), Ptr<Socket> socket, Ptr<Packet> packet, Address address)
        {
            return MakeEvent(f, socket, packet, address);
        }
    }
    #endif
""")

In [ ]:
import sys
class EchoServer(ns.Application):
    LOGGING = True
    ECHO_PORT = 1234 # Application default listening port
    socketToInstanceDict = {}

    def __init__(self, node: ns.Node, port=ECHO_PORT):
        super().__init__() # Calls ns.Application constructor
        self.__python_owns__ = False  # Allow C++ to destroy this object when Simulator::Destroy gets called
        self.port = port # Server listening port

        # Create an UDP socket
        self.m_socket = ns.Socket.CreateSocket(node,
                                               ns.TypeId.LookupByName("ns3::UdpSocketFactory"))
        # Bind that socket to a specific port and IP addresses to listen
        self.m_socket.Bind(ns.InetSocketAddress(ns.Ipv4Address.GetAny(), self.port).ConvertTo())

        # Creates the callback to handle packet reception using a static function of the EchoServerClass
        # (This is a workaround for the application to work from Python)
        self.m_socket.SetRecvCallback(ns.make_rx_callback(EchoServer._Receive))

        # Registers the socket as a dictionary key and associate it to this application instance
        EchoServer.socketToInstanceDict[self.m_socket] = self

    def __del__(self):
        # Remove the instance entry on the class dictioanry when the object is destroyed
        del EchoServer.socketToInstanceDict[self.m_socket]

    def Send(self, packet: ns.Packet, address: ns.Address) -> None:
        # Function that sends the packet to a target address.
        self.m_socket.SendTo(packet, 0, address)
        if EchoServer.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s server sent {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)

    def Receive(self):
        # Function that receives a packet and the address of the sender
        address = ns.Address()
        packet = self.m_socket.RecvFrom(address)
        if EchoServer.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s server received {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)
        # Schedule the Send function to send the packet back to its sender after 1 second
        event = ns.pythonMakeEventSend(EchoServer._Send, self.m_socket, packet, address)
        ns.Simulator.Schedule(ns.Seconds(1), event)

    @staticmethod
    def _Send(socket: ns.Socket, packet: ns.Packet, address: ns.Address):
        # Static function that identifies the EchoServer instance that is
        # supposed to send a packet and calls it to send
        instance = EchoServer.socketToInstanceDict[socket]
        instance.Send(packet, address)
        pass

    @staticmethod
    def _Receive(socket: ns.Socket) -> None:
        # Static function that identifies the EchoServer instance that is
        # supposed to receive a packet and calls it to receive
        instance = EchoServer.socketToInstanceDict[socket]
        instance.Receive()
        pass

In [ ]:
echoServer = EchoServer(nodes.Get(1))
nodes.Get(1).AddApplication(echoServer)

serverApps = ns.ApplicationContainer()
serverApps.Add(echoServer)
serverApps.Start(ns.Seconds(1.0))
serverApps.Stop(ns.Seconds(10.0))

address = interfaces.GetAddress(1).ConvertTo()

ns.LogComponentEnable("UdpEchoClientApplication", ns.LOG_LEVEL_INFO)
echoClient = ns.UdpEchoClientHelper(address, EchoServer.ECHO_PORT)
echoClient.SetAttribute("MaxPackets", ns.UintegerValue(10))
echoClient.SetAttribute("Interval", ns.TimeValue(ns.Seconds(1.0)))
echoClient.SetAttribute("PacketSize", ns.UintegerValue(101))

clientApps = echoClient.Install(nodes.Get(0))
clientApps.Start(ns.Seconds(2.0))
clientApps.Stop(ns.Seconds(10.0))

ns.Simulator.Run()
ns.Simulator.Destroy()
ns.FatalImpl.FlushStreams() # Make sure the pcaps are written

In [ ]:
def print_packets_with_tshark(packet_i):
    import pyshark
    cap = pyshark.FileCapture('application-0-0.pcap')
    i = 0
    for packet in cap:
        if packet_i == i:
            print(packet)
            break
        i += 1
    cap.close()

# Jupyter shenanigans with Pyshark asyncio code
import concurrent.futures
def exec_async(func, *args, **kwargs):
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(func, *args, **kwargs)
    return future.result()
result = exec_async(print_packets_with_tshark, packet_i=1)

In [ ]:
result = exec_async(print_packets_with_tshark, packet_i=2)

**Audio**

In [ ]:
import sys
from hashlib import sha256 as sha

ns.cppyy.cppdef("""
    #ifndef StreamingAppSendBlock
    #define StreamingAppSendBlock
    namespace ns3
    {
        EventImpl* pythonMakeEventSendBlock(void (*f)(Ptr<Socket>, Address&), Ptr<Socket> socket, Address address)

        {
            return MakeEvent(f, socket, address);
        }
    }
    #endif
""")
samples_per_block = 200

class StreamingServer(ns.Application):
    LOGGING = False
    STREAMING_PORT = 2345  # Application default listening port
    socketToInstanceDict = {}

    def __init__(self, node: ns.Node, port=STREAMING_PORT, wav_file=None):
        if not wav_file:
            raise Exception("No wav file was specified")
        import os
        if not os.path.exists(wav_file):
            raise Exception("File does not exist", wav_file)
        super().__init__()  # Calls ns.Application constructor
        self.__python_owns__ = False  # Allow C++ to destroy this object when Simulator::Destroy gets called
        self.port = port  # Server listening port
        # Create an UDP socket
        self.m_socket = ns.Socket.CreateSocket(node,
                                               ns.TypeId.LookupByName("ns3::UdpSocketFactory"))
        # Bind that socket to a specific port and IP addresses to listen
        self.m_socket.Bind(ns.InetSocketAddress(ns.Ipv4Address.GetAny(), self.port).ConvertTo())

        # Creates the callback to handle packet reception using a static function of the StreamingServerClass
        # (This is a workaround for the application to work from Python)
        self.m_socket.SetRecvCallback(ns.make_rx_callback(StreamingServer._Receive))

        # Registers the socket as a dictionary key and associate it to this application instance
        StreamingServer.socketToInstanceDict[self.m_socket] = self

        # Load the wav file
        import wave
        wav_file = wave.open(wav_file, "rb")

        # Read blocks of 200 samples
        self.blocks_to_play = []
        data = None
        while data != b"":
            data = wav_file.readframes(samples_per_block)
            self.blocks_to_play.append(data)

        self.blocks_to_play = self.blocks_to_play[2500:7500]

    def __del__(self):
        # Remove the instance entry on the class dictionary when the object is destroyed
        del StreamingServer.socketToInstanceDict[self.m_socket]

    def SendBlock(self, address: ns.Address):
        # Check if list of blocks to transmit ended
        if len(self.blocks_to_play) == 0:
            return

        # Get a block
        block_to_transmit = self.blocks_to_play[0]
        self.blocks_to_play.pop(0)
        #print("sending ", sha(block_to_transmit).hexdigest())
        # Encode bytestream into a ns-3 packet
        packet_with_stream_block = ns.Packet(bytearray(block_to_transmit), len(block_to_transmit))
        packet_with_stream_block.__python_owns__ = False

        self.Send(packet_with_stream_block, address)

        # Re-schedule event until the blocks list run out
        event = ns.pythonMakeEventSendBlock(self._SendBlock, self.m_socket, address)
        ns.Simulator.Schedule(ns.MilliSeconds(60), event)

    def Send(self, packet: ns.Packet, address: ns.Address) -> None:
        # Function that sends the packet to a target address.
        # Assemble a new Packet containing the streamed block
        # Send the Packet to the address
        self.m_socket.SendTo(packet, 0, address)
        if StreamingServer.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s server sent {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)
            if ns.Simulator.Now().GetSeconds() > 5:
                StreamingServer.LOGGING = False

    def Receive(self):
        # Function that receives a packet and the address of the sender
        address = ns.Address()
        packet = self.m_socket.RecvFrom(address)
        if StreamingServer.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s server received {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)
            if ns.Simulator.Now().GetSeconds() > 5:
                StreamingServer.LOGGING = False

        # Whoever sends a packet to server will start receiving the stream
        # Start streaming blocks after 1 second
        event = ns.pythonMakeEventSendBlock(self._SendBlock, self.m_socket, address)
        ns.Simulator.Schedule(ns.MilliSeconds(60), event)

    @staticmethod
    def _SendBlock(socket: ns.Socket, address: ns.Address):
        # Static function that identifies the StreamingServer instance that is
        # supposed to send a packet and calls it to send
        instance = StreamingServer.socketToInstanceDict[socket]
        instance.SendBlock(address)

    @staticmethod
    def _Send(socket: ns.Socket, packet: ns.Packet, address: ns.Address):
        # Static function that identifies the StreamingServer instance that is
        # supposed to send a packet and calls it to send
        instance = StreamingServer.socketToInstanceDict[socket]
        instance.Send(packet, address)
        pass

    @staticmethod
    def _Receive(socket: ns.Socket) -> None:
        # Static function that identifies the StreamingServer instance that is
        # supposed to receive a packet and calls it to receive
        instance = StreamingServer.socketToInstanceDict[socket]
        instance.Receive()
        pass


class StreamingClient(ns.Application):
    LOGGING = False
    socketToInstanceDict = {}

    def __init__(self, node: ns.Node, serverAddress: ns.Ipv4Address, port=StreamingServer.STREAMING_PORT,
                 wav_file=None):
        if not wav_file:
            raise Exception("No wav file was specified")
        import os
        if not os.path.exists(wav_file):
            raise Exception("File does not exist", wav_file)
        super().__init__()  # Calls ns.Application constructor
        self.__python_owns__ = False  # Allow C++ to destroy this object when Simulator::Destroy gets called
        self.port = port  # Server listening port
        # Create an UDP socket
        self.m_socket = ns.Socket.CreateSocket(node,
                                               ns.TypeId.LookupByName("ns3::UdpSocketFactory"))

        # Creates the callback to handle packet reception using a static function of the StreamingClient class
        # (This is a workaround for the application to work from Python)
        self.m_socket.SetRecvCallback(ns.make_rx_callback(StreamingClient._Receive))

        # Registers the socket as a dictionary key and associate it to this application instance
        StreamingClient.socketToInstanceDict[self.m_socket] = self

        # Schedule the Send function to send the packet back to its sender after 1 second
        packet = ns.Packet(0)
        packet.__python_owns__ = False
        address = ns.InetSocketAddress(serverAddress, port).ConvertTo()
        event = ns.pythonMakeEventSend(StreamingClient._Send, self.m_socket, packet, address)
        ns.Simulator.Schedule(ns.Seconds(1), event)

        import wave
        self.output_audio = wave.open("audio_out.wav", "wb")
        with wave.open(wav_file, "rb") as f:
            self.output_audio.setsampwidth(f.getsampwidth())
            self.output_audio.setframerate(f.getframerate())
            self.output_audio.setnchannels(f.getnchannels())

    def __del__(self):
        # Remove the instance entry on the class dictionary when the object is destroyed
        del StreamingClient.socketToInstanceDict[self.m_socket]
        self.output_audio.close()

    def Receive(self):
        # Function that receives a packet and the address of the sender
        address = ns.Address()
        packet = self.m_socket.RecvFrom(address)
        if StreamingClient.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s client received {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)
            if ns.Simulator.Now().GetSeconds() > 5:
                StreamingClient.LOGGING = False

        # Extract bytestream from Packet and play
        from ctypes import c_uint8

        size = packet.__deref__().GetSize()
        contents_bytes_array = c_uint8*size
        contents = contents_bytes_array()
        packet.__deref__().CopyData(contents, size)
        contents = bytes(contents)
        #print("received ", sha(contents).hexdigest())

        self.output_audio.writeframes(contents)

    @staticmethod
    def _Receive(socket: ns.Socket) -> None:
        # Static function that identifies the StreamingClient instance that is
        # supposed to receive a packet and calls it to receive
        instance = StreamingClient.socketToInstanceDict[socket]
        instance.Receive()
        pass

    def Send(self, packet: ns.Packet, address: ns.Address) -> None:
        # Function that sends the packet to a target address.
        # Assemble a new Packet containing the streamed block
        # Send the Packet to the address
        self.m_socket.SendTo(packet, 0, address)
        if StreamingClient.LOGGING:
            inetAddress = ns.InetSocketAddress.ConvertFrom(address)
            print("At time +{s}s client sent {b} bytes from {ip} port {port}"
                  .format(s=ns.Simulator.Now().GetSeconds(),
                          b=packet.__deref__().GetSize(),
                          ip=inetAddress.GetIpv4(),
                          port=inetAddress.GetPort()),
                  file=sys.stderr,
                  flush=True)
            if ns.Simulator.Now().GetSeconds() > 5:
                StreamingClient.LOGGING = False

    @staticmethod
    def _Send(socket: ns.Socket, packet: ns.Packet, address: ns.Address):
        # Static function that identifies the StreamingClient instance that is
        # supposed to send a packet and calls it to send
        instance = StreamingClient.socketToInstanceDict[socket]
        instance.Send(packet, address)
        pass

In [ ]:
from IPython.display import Audio
from IPython.core.display import display

wav_file = "./200_the_application_layer/richard-michael-walker_titan.wav"
display(Audio(url=wav_file, embed=False, autoplay=False))

def run_application(lossModel=False, byteErrorRate=0):
    nodes = ns.NodeContainer()
    nodes.Create(2)

    pointToPoint = ns.PointToPointHelper()
    pointToPoint.SetDeviceAttribute("DataRate", ns.StringValue("5Mbps"))
    pointToPoint.SetChannelAttribute("Delay", ns.StringValue("2ms"))

    devices = pointToPoint.Install(nodes)

    if lossModel:
        for i in range(devices.GetN()):
            em = ns.CreateObject[ns.RateErrorModel]()
            em.__deref__().SetAttribute("ErrorRate", ns.DoubleValue(byteErrorRate))
            devices.Get(i).__deref__().SetAttribute("ReceiveErrorModel", ns.PointerValue(em))

    stack = ns.InternetStackHelper()
    stack.Install(nodes)

    address = ns.Ipv4AddressHelper()
    address.SetBase(ns.Ipv4Address("10.1.1.0"),
                    ns.Ipv4Mask("255.255.255.0"))

    interfaces = address.Assign(devices)

    streamServer = StreamingServer(nodes.Get(1), wav_file=wav_file)
    nodes.Get(1).AddApplication(streamServer)
    serverApps = ns.ApplicationContainer()
    serverApps.Add(streamServer)
    serverApps.Start(ns.Seconds(1.0))
    serverApps.Stop(ns.Seconds(10.0))

    address = interfaces.GetAddress(1)

    streamClient = StreamingClient(nodes.Get(0), address, StreamingServer.STREAMING_PORT, wav_file=wav_file)
    clientApps = ns.ApplicationContainer()
    clientApps.Add(streamClient)
    clientApps.Start(ns.Seconds(2.0))
    clientApps.Stop(ns.Seconds(10.0))

    ns.Simulator.Run()
    ns.Simulator.Destroy()

    # Add widget with modified audio
    display(Audio("audio_out.wav", autoplay=False))

In [ ]:
run_application()

In [ ]:
run_application(lossModel=True, byteErrorRate=0.02)